## Pull Data From URL and Make Your Dataframe

In [47]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab *already downloaded
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

ModuleNotFoundError: No module named 'folium'

In [48]:
url='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050'

df = pd.read_html(url)

df_postcodes=df[0]

print('imported dataframe has', df_postcodes['Postcode'].count(), "postcodes entriess")

df_postcodes.head(10)
#df_postcodes.dtypes

imported dataframe has 287 postcodes entriess


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


Clean the data - Get rid of N/A Boroughs and for any N/A Nieghbhorhoods, add the Borough name to Nieghborhoods

In [49]:
#df_postcodes["Borough"].replace("nan", "Not assigned", inplace=True)
#df_postcodes["Neighbourhood"].replace("nan", "Not assigned", inplace=True)


df_postcodes = df_postcodes[df_postcodes["Borough"] != "Not assigned"]
#df_postcodes.dropna(subset=["Borough"], axis=0, inplace=True)

#Replace the N/A values of Neighborhood with the Borough Value
df_postcodes["Neighbourhood"].replace("Not assigned", df_postcodes["Borough"], inplace=True)

df_postcodes.head(20)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [50]:
#Now combine all the neighbourhoods with the same postal codes
df = df_postcodes.groupby(["Postcode","Borough"])["Neighbourhood"].apply(list)
df = df.sample(frac=1).reset_index()
df["Neighbourhood"] = df["Neighbourhood"].str.join(', ')
df.head(10)

,Postcode,Borough,Neighbourhood
0,M7A,Downtown Toronto,Queen's Park
1,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
2,M1N,Scarborough,"Birch Cliff, Cliffside West"
3,M4E,East Toronto,The Beaches
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
5,M3N,North York,Downsview Northwest
6,M6C,York,Humewood-Cedarvale
7,M2M,North York,"Newtonbrook, Willowdale"
8,M4G,East York,Leaside
9,M6P,West Toronto,"High Park, The Junction South"


In [51]:
df.shape

(103, 3)